In [ ]:
### This is mostly a demo on different modules / classes available in nba_api using steph as an example

In [21]:
# Imports
from nba_api.stats.static import players
from nba_api.stats.endpoints import (
    PlayerGameLog,
    BoxScoreTraditionalV2,
    BoxScoreAdvancedV2,
    DraftCombinePlayerAnthro,
    CommonPlayerInfo
)
import pandas as pd

In [22]:
# Get the ID for the player so we can have portable lookups
pid = players.find_players_by_full_name("Stephen Curry")[0]["id"]

print(pid)
logs = PlayerGameLog(
    player_id=pid,
    season="2016-17",
    season_type_all_star="Playoffs"
).get_data_frames()[0]
# print(logs)

finals_logs = logs[logs["MATCHUP"].str.contains("CLE")] # GSW played CLE in the finals in 2017
print(finals_logs)

201939
  SEASON_ID  Player_ID     Game_ID     GAME_DATE      MATCHUP WL  MIN  FGM  \
0     42016     201939  0041600405  Jun 12, 2017  GSW vs. CLE  W   41   10   
1     42016     201939  0041600404  Jun 09, 2017    GSW @ CLE  L   38    4   
2     42016     201939  0041600403  Jun 07, 2017    GSW @ CLE  W   39    8   
3     42016     201939  0041600402  Jun 04, 2017  GSW vs. CLE  W   36    7   
4     42016     201939  0041600401  Jun 01, 2017  GSW vs. CLE  W   34   11   

   FGA  FG_PCT  ...  DREB  REB  AST  STL  BLK  TOV  PF  PTS  PLUS_MINUS  \
0   20   0.500  ...     4    6   10    3    0    4   3   34           3   
1   13   0.308  ...     4    5   10    2    0    4   2   14         -24   
2   19   0.421  ...     8   13    6    2    0    1   3   26          11   
3   17   0.412  ...     8   10   11    1    0    8   3   32          21   
4   22   0.500  ...     5    6   10    3    0    2   3   28          20   

   VIDEO_AVAILABLE  
0                1  
1                1  
2         

In [23]:
game_id = finals_logs.iloc[0]["Game_ID"]
print(f"Using Finals Game_ID: {game_id}\n")

Using Finals Game_ID: 0041600405



In [24]:
player_stats = BoxScoreTraditionalV2(game_id=game_id)
stats_df = player_stats.get_data_frames()[2]
stats_df

,GAME_ID,TEAM_ID,TEAM_NAME,TEAM_ABBREVIATION,TEAM_CITY,STARTERS_BENCH,MIN,FGM,FGA,FG_PCT,...,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TO,PF,PTS
0,0041600405,1610612739,Cavaliers,CLE,Cleveland,Starters,188:36,45,79,0.570,...,0.619,10,26,36,20,5,5,9,11,113
1,0041600405,1610612744,Warriors,GSW,Golden State,Starters,168:44,31,64,0.484,...,0.800,10,22,32,23,7,1,10,18,94
2,0041600405,1610612739,CLE,Cleveland,cavaliers,Bench,51:24,2,9,0.222,...,1.000,2,2,4,2,1,0,5,11,7
3,0041600405,1610612744,GSW,Golden State,warriors,Bench,71:16,15,26,0.577,...,1.000,3,7,10,4,1,1,3,6,35


In [26]:
player_stats = BoxScoreTraditionalV2(game_id=game_id)
stats_df = player_stats.get_data_frames()[0]
stats_df.sample(5)

,GAME_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_CITY,PLAYER_ID,PLAYER_NAME,NICKNAME,START_POSITION,COMMENT,MIN,...,OREB,DREB,REB,AST,STL,BLK,TO,PF,PTS,PLUS_MINUS
25,0041600405,1610612744,GSW,Golden State,201580,JaVale McGee,JaVale,,DNP - Coach's Decision,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,0041600405,1610612739,CLE,Cleveland,202697,Iman Shumpert,Iman,,,3:45,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,3.0,0.0,-9.0
3,0041600405,1610612739,CLE,Cleveland,2747,JR Smith,JR,G,,40:49,...,0.0,3.0,3.0,1.0,0.0,2.0,0.0,2.0,25.0,-2.0
23,0041600405,1610612744,GSW,Golden State,203546,Ian Clark,Ian,,DNP - Coach's Decision,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11,0041600405,1610612739,CLE,Cleveland,2592,James Jones,James,,DNP - Coach's Decision,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [27]:
anthro = DraftCombinePlayerAnthro().get_data_frames()[0]
anthro_data = anthro[anthro["PLAYER_ID"] == pid][
    ["PLAYER_NAME", "HEIGHT_WO_SHOES", "HEIGHT_W_SHOES", "WEIGHT", "WINGSPAN"]
]

if anthro_data.empty:
    info = CommonPlayerInfo(player_id=pid).common_player_info.get_data_frame().iloc[0]
    anthro_data = pd.DataFrame([{
        "PLAYER_NAME": info["DISPLAY_FIRST_LAST"],
        "HEIGHT_W_SHOES": info["HEIGHT"],
        "WEIGHT": info["WEIGHT"],
        "WINGSPAN": None
    }])
print(anthro_data)

     PLAYER_NAME HEIGHT_W_SHOES WEIGHT WINGSPAN
0  Stephen Curry            6-2    185     None
